# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 1 2022 10:58AM,243144,10,0085917009,ISDIN Corporation,Released
1,Aug 1 2022 10:58AM,243144,10,0085917284,ISDIN Corporation,Released
2,Aug 1 2022 10:58AM,243144,10,0085917285,ISDIN Corporation,Released
3,Aug 1 2022 10:58AM,243144,10,0085917288,ISDIN Corporation,Released
4,Aug 1 2022 10:58AM,243144,10,0085917289,ISDIN Corporation,Released
5,Aug 1 2022 10:58AM,243144,10,0085917291,ISDIN Corporation,Released
6,Aug 1 2022 10:58AM,243144,10,0085917296,ISDIN Corporation,Released
7,Aug 1 2022 10:58AM,243144,10,0085917303,ISDIN Corporation,Released
8,Aug 1 2022 10:58AM,243144,10,0085917307,ISDIN Corporation,Released
9,Aug 1 2022 10:58AM,243144,10,0085917308,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,243139,Released,50
22,243141,Released,1
23,243142,Released,42
24,243144,Released,64
25,243145,Released,48


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
243139,NaN,50.0
243141,NaN,1.0
243142,NaN,42.0
243144,NaN,64.0
243145,NaN,48.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
243085,0.0,1.0
243091,0.0,1.0
243101,0.0,1.0
243108,26.0,6.0
243110,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
243085,0,1
243091,0,1
243101,0,1
243108,26,6
243110,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,243085,0,1
1,243091,0,1
2,243101,0,1
3,243108,26,6
4,243110,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,243085,,1
1,243091,,1
2,243101,,1
3,243108,26,6
4,243110,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation
64,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc."
112,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc."
113,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc."
114,Aug 1 2022 10:55AM,243085,20,"ACI Healthcare USA, Inc."
115,Aug 1 2022 10:53AM,243138,20,Alumier Labs Inc.
123,Aug 1 2022 10:46AM,243142,10,ISDIN Corporation
165,Aug 1 2022 10:35AM,243141,19,ACG North America LLC
166,Aug 1 2022 10:32AM,243139,10,ISDIN Corporation
216,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,,64
1,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",,48
2,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",,1
3,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",,1
4,Aug 1 2022 10:55AM,243085,20,"ACI Healthcare USA, Inc.",,1
5,Aug 1 2022 10:53AM,243138,20,Alumier Labs Inc.,1,7
6,Aug 1 2022 10:46AM,243142,10,ISDIN Corporation,,42
7,Aug 1 2022 10:35AM,243141,19,ACG North America LLC,,1
8,Aug 1 2022 10:32AM,243139,10,ISDIN Corporation,,50
9,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,,46


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,64,
1,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",48,
2,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",1,
3,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",1,
4,Aug 1 2022 10:55AM,243085,20,"ACI Healthcare USA, Inc.",1,
5,Aug 1 2022 10:53AM,243138,20,Alumier Labs Inc.,7,1
6,Aug 1 2022 10:46AM,243142,10,ISDIN Corporation,42,
7,Aug 1 2022 10:35AM,243141,19,ACG North America LLC,1,
8,Aug 1 2022 10:32AM,243139,10,ISDIN Corporation,50,
9,Aug 1 2022 10:14AM,243135,10,ISDIN Corporation,46,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,64,
1,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",48,
2,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",1,
3,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",1,
4,Aug 1 2022 10:55AM,243085,20,"ACI Healthcare USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,64.0,NaN
1,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",48.0,NaN
2,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",1.0,NaN
3,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",1.0,NaN
4,Aug 1 2022 10:55AM,243085,20,"ACI Healthcare USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 1 2022 10:58AM,243144,10,ISDIN Corporation,64.0,0.0
1,Aug 1 2022 10:58AM,243145,20,"ACI Healthcare USA, Inc.",48.0,0.0
2,Aug 1 2022 10:57AM,243101,20,"ACI Healthcare USA, Inc.",1.0,0.0
3,Aug 1 2022 10:56AM,243091,20,"ACI Healthcare USA, Inc.",1.0,0.0
4,Aug 1 2022 10:55AM,243085,20,"ACI Healthcare USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3160640,275.0,2.0
16,243123,2.0,0.0
19,972505,8.0,27.0
20,1215560,58.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3160640,275.0,2.0
1,16,243123,2.0,0.0
2,19,972505,8.0,27.0
3,20,1215560,58.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,275.0,2.0
1,16,2.0,0.0
2,19,8.0,27.0
3,20,58.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,275.0
1,16,Released,2.0
2,19,Released,8.0
3,20,Released,58.0
4,10,Executing,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,16,19,20
Status,,,,
Executing,2.0,0.0,27.0,1.0
Released,275.0,2.0,8.0,58.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,16,19,20
0,Executing,2.0,0.0,27.0,1.0
1,Released,275.0,2.0,8.0,58.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,16,19,20
0,Executing,2.0,0.0,27.0,1.0
1,Released,275.0,2.0,8.0,58.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()